In [ ]:
# @REMOVE-FROM-TEMPLATE
from plaster.tools.ipynb_helpers.displays import restart_kernel; restart_kernel()

In [1]:
# @IMPORT-MERGE
import numpy as np
import pandas as pd
import itertools
from plaster.tools.log.log import error, debug
from plaster.run.job import JobResult
from plaster.run.run import RunResult
from plaster.run.plots import plots_sigproc
from plaster.run.plots import plots_dev
from plaster.run.sigproc_v2 import sigproc_v2_worker as worker
from plaster.tools.image.coord import WH, XY, roi_shift, clip2d
from plaster.tools.zplots import zplots
from plaster.tools.schema import check
from plaster.tools.image import imops
from plaster.tools.zap import zap
from plaster.tools.utils import utils
from plaster.tools.calibration.calibration import Calibration
from plaster.run.sigproc_v2.sigproc_v2_params import SigprocV2Params
from plaster.run.sigproc_v2.reg_psf import RegPSF
from plaster.tools.ipynb_helpers.displays import hd
z = zplots.setup()

Matplotlib created a temporary config/cache directory at /root/.config/matplotlib because the default path (/tmp/matplotlib-hm5_4x5a) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Loading BokehJS ...

In [2]:
# @REMOVE-FROM-TEMPLATE
from plumbum import local
job = JobResult("/erisyon/internal/jobs_folder/calib_abbe7_1t")
run = job.runs[0]

In [3]:
q = run.ims_import.qualities()
z.hist(q.quality.values)
#q[q.quality > 300].field_i.unique()

In [4]:
# z.im(run.ims_import.ims[222, 0, 0])

# PSF

In [4]:
for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")
    reg_psf = run.sigproc_v2.calib.psfs(ch_i=ch_i)
    check.t(reg_psf, RegPSF)
    psf_ims = reg_psf.render()
    plots_sigproc.plot_psfs(psf_ims, scale=3.0, f_title=f"ch_i={ch_i}", _noaxes=True, _notools=True)

# Illumination Balance

In [5]:
for ch_i in range(run.ims_import.n_channels):
    with z(_cols=6):
        key = f"regional_illumination_balance.instrument_channel[{ch_i}]"
        val = run.sigproc_v2.calib.get(key)
        if val is not None:
            hd("h2", f"Channel {ch_i}")
            bal = np.array(val)
            z.im(1.0 / bal)
        else:
            print(f"No illumination balance on channel {ch_i}")